In [ ]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
import math
import time

2024-11-04 20:06:41.970446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# number of parts to train on
num_data = 15
# max length of each part
max_sequence_length = 150

# read from melodyData.txt
with open('out/melodyData.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

# one-hot encoding
encodings = {}
encodingIndex = 0
for part in parts[:num_data]:
    for note in part[:max_sequence_length]:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

# decoder constructed by reversing one-hot encoding
decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a
data_encoded = []
for part in parts[:num_data]:
    data_encoded.append([encodings[note] for note in part[:max_sequence_length]])

num_data = min(num_data, len(data_encoded))

Number of parts:11554


In [9]:
# one-hot encode the data
def generate_data(data_encoded, encodings):
    X = []
    Y = []
    # given data_encoded, generate training data by looping 
    for i in range(len(data_encoded)):
        currentX = []
        currentY = []
        for j in range(len(data_encoded[i])-1):
            currentX.append(data_encoded[i][j])
            currentY.append(data_encoded[i][j+1])
        X.append(currentX)
        Y.append(currentY)

    X_onehot = []
    for seq in X:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        X_onehot.append(onehot)
    X = X_onehot
    
    Y_onehot = []
    for seq in Y:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
            
        Y_onehot.append(onehot)
    Y = Y_onehot

    return X, Y

X_train, Y_train = generate_data(data_encoded[:math.floor(num_data*0.7)], encodings)
#X_test, Y_test = generate_data(data_encoded[10+math.floor(num_data*0.8)], encodings)
X_test, Y_test = generate_data(data_encoded[math.floor(num_data*0.7):], encodings)

# pads sequences so we can convert to numpy arrays
X_train = tf.keras.utils.pad_sequences(X_train, padding='pre')
Y_train = tf.keras.utils.pad_sequences(Y_train, padding='pre')

In [10]:
print (X_train.shape)
print (Y_train.shape)

(10, 149, 98)
(10, 149, 98)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(32, stateful=True, batch_input_shape=(1, None, len(encodings))),
    #tf.keras.layers.InputLayer(input_shape=(None, len(encodings))),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

timeTaken = time.time()

# Train the model one time step at a time
epochs = 1
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    for i, sequence in enumerate(X_train):
        print (f'Sequence {i}')
        # Reset states at the beginning of each sequence
        model.reset_states() 
        # TODO: Train on entire sequence at once
        for t in range(sequence.shape[0]): 
            x = sequence[t:t+1].reshape(1, 1, len(encodings))
            #x = sequence[t:t+1].reshape(1, len(encodings))
            y = Y_train[i][t:t+1] 
            #print (decodings[np.argmax(x)], decodings[np.argmax(y)])
            loss = model.train_on_batch(x, y) 

print (f'Training time in seconds: {time.time() - timeTaken}')
# 34.8 seconds

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, None, 64)          6336      
                                                                 
 dense_42 (Dense)            (None, None, 98)          6370      
                                                                 
Total params: 12706 (49.63 KB)
Trainable params: 12706 (49.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/1
Sequence 0
Sequence 1
Sequence 2
Sequence 3
Sequence 4
Sequence 5
Sequence 6
Sequence 7
Sequence 8
Sequence 9
Training time in seconds: 23.89521098136902


In [67]:
# count the number of appearances of each item, DEBUG ONLY
# don't remove for now
"""from collections import defaultdict as dd

appearances = dd(int)

total = 0
c = 0
for i in range(len(X_train)):
    for j in range(X_train[i].shape[0]):
        total += 1
        appearances[decodings[np.argmax(X_train[i][j])]] += 1
        #if np.argmax(X_train[i][j]) == encodings[(13, '2.00')]:
            #c += 1
# sort appearances
appearances = dict(sorted(appearances.items(), key=lambda item: item[1], reverse=True))
for i in appearances:
    print (appearances[i], appearances[i]/total, i)
print (encodings[(0, '3.00')])"""

"from collections import defaultdict as dd\n\nappearances = dd(int)\n\ntotal = 0\nc = 0\nfor i in range(len(X_train)):\n    for j in range(X_train[i].shape[0]):\n        total += 1\n        appearances[decodings[np.argmax(X_train[i][j])]] += 1\n        #if np.argmax(X_train[i][j]) == encodings[(13, '2.00')]:\n            #c += 1\n# sort appearances\nappearances = dict(sorted(appearances.items(), key=lambda item: item[1], reverse=True))\nfor i in appearances:\n    print (appearances[i], appearances[i]/total, i)\nprint (encodings[(0, '3.00')])"

In [ ]:
# tests model accuracy

correct = 0.0
total = 0
for i in range(len(X_test)):
    # reset model
    model.reset_states()
    for j in range(len(X_test[i])):
        # feed it first note, get next note repeatedly
        note = X_test[i][j:j+1]
        pred = model.predict(np.array([note]), verbose=0) 
        #print (np.max(pred)) # 
        # output, correct note, confidence
        print (decodings[np.argmax(pred)], decodings[np.argmax(Y_test[i][j])], np.max(pred))
        if (np.argmax(pred) == np.argmax(Y_test[i][j])):
            correct += 1
        total += 1
print(correct/total)

88
(24, '1.00') (7, '0.75') 0.103982896
0.0
80
(0, '0.25') (0, '0.25') 0.14935021
0.5
88
(24, '1.00') (10, '1.00') 0.103982896
0.3333333333333333
80
(0, '0.25') (7, '0.50') 0.09433649
0.25
80
(0, '0.25') (3, '0.25') 0.09968295
0.2
80
(0, '0.25') (0, '0.25') 0.14534539
0.3333333333333333
88
(24, '1.00') (17, '0.50') 0.103982896
0.2857142857142857
88
(24, '1.00') (20, '0.50') 0.095512375
0.25
80
(0, '0.25') (17, '1.00') 0.09921778
0.2222222222222222
88
(24, '1.00') (0, '0.50') 0.10171458
0.2
88
(24, '1.00') (12, '0.50') 0.11404161
0.18181818181818182
66
(12, '0.50') (8, '0.50') 0.108479515
0.16666666666666666
80
(0, '0.25') (5, '0.50') 0.11339265
0.15384615384615385
80
(0, '0.25') (5, '1.00') 0.08445969
0.14285714285714285
88
(24, '1.00') (7, '1.00') 0.11756744
0.13333333333333333
80
(0, '0.25') (8, '1.00') 0.10539391
0.125
87
(19, '1.00') (12, '1.00') 0.10129852
0.11764705882352941
88
(24, '1.00') (17, '1.00') 0.09087128
0.1111111111111111
88
(24, '1.00') (8, '0.25') 0.09087128
0.105263

KeyboardInterrupt: 

In [58]:
import copy
# given a sequence, generate the next note (up to ten times)

model.reset_states()

# generate the first 10 notes after we input 1

#pred = model.predict(encodings[1])
one_hot = np.zeros((len(encodings)))
one_hot[1] = 1
print (np.array([one_hot]))
pred = model.predict(np.array([[one_hot]]))
for i in range(10):
    one_hot = np.zeros((len(encodings)))
    one_hot[np.argmax(pred)] = 1
    pred = model.predict(np.array([[one_hot]]))
    print (decodings[np.argmax(pred)])


NameError: name 'model' is not defined